##### **Setup**
##### Download report from BCC
1. Go to the web bank client
2. Under each currency account choose "Выписка"
3. After choosing a period -> "Отправить на почту" 


#### to-do
- add flag for not using in analysis
- check categories per month
- add analitics for cashback (and when is is added to the account)
- top-10 biggest spendings in every category
- check subscriptions
- separate preparation of data with analytics

In [52]:
#imports
import pandas as pd
import re

pd.set_option('display.max_colwidth', None)

In [67]:
#parse euro_acc table
df_euro = pd.read_html('euro_bcc_export.html')[2]
#parse tenge_acc table
df_tenge = pd.read_html('tenge_bcc_export.html')[2]

### Preparing BCC Euro DF

In [70]:
#renaming headers
df_euro = df_euro.rename(
    columns={
        '№ п/п': 'id',
        'Дата': 'record_dt',
        'Дебет': 'sum',
        'Кредит': 'sum_temp',
        'Назначение': 'details', }
)

#changing types
df_euro['id'] = df_euro['id'].astype('int')

df_euro['sum'] = df_euro['sum'].str.replace(r'\s+', '', regex=True)
df_euro['sum'] = pd.to_numeric(df_euro['sum'])

df_euro['sum_temp'] = df_euro['sum_temp'].str.replace(r'\s+', '', regex=True)
df_euro['sum_temp'] = pd.to_numeric(df_euro['sum_temp'])

df_euro['record_dt'] = pd.to_datetime(df_euro['record_dt'], format='%d.%m.%Y')

,id,record_dt,sum,sum_temp,details
0,1,2024-01-01,4.25,NaN,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312290754, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1318| BCC: 1; IPS: 1.1142"
1,2,2024-01-01,NaN,0.09,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 4.25 EUR"
2,3,2024-01-01,NaN,0.11,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 5.59 EUR"
3,4,2024-01-01,5.59,NaN,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312291925, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1306| BCC: 1; IPS: 1.1142"
4,5,2024-01-01,NaN,0.13,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 6.59 EUR"
...,...,...,...,...,...
4052,4053,2024-12-30,7.00,NaN,"Retail. Номер устройства в ПЦ, 27.12.2024 00:00:00, PRT, LISBOA, FABRICA COFFE, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0586| BCC: 1; IPS: 1.0434"
4053,4054,2024-12-30,7.60,NaN,"Retail. Номер устройства в ПЦ, 27.12.2024 00:00:00, PRT, LISBOA, FABRICA COFFE, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0592| BCC: 1; IPS: 1.0434"
4054,4055,2024-12-30,NaN,1098.48,"Покупка иностранной валюты клиентом розницы, Коренблюм Герман Владимирович, БИН 920404050799. Заявка №11013165686. Дата вал. 30.12.2024. Покупка EUR, продажа KZT. Курс сделки 546.21. Учетный курс 543.67. Конвертация со счета KZ348562204134322019 на счет KZ938562204234322234 через систему BCC.KZ"
4055,4056,2024-12-31,4.80,NaN,"Retail. Номер устройства в ПЦ, 28.12.2024 00:00:00, PRT, LISBOA, REST LULLY 1661, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0604| BCC: 1; IPS: 1.0445"


In [71]:
#moving all sums to one column
def move_income_to_sum_column(row):
    if pd.isna(row['sum']):
        row['sum'] = row['sum_temp']
    if pd.isna(row['sum_temp']):
        row['sum'] = -row['sum']
    return row

df_euro = df_euro.apply(move_income_to_sum_column, axis=1)
df_euro = df_euro.drop('sum_temp', axis=1)

,id,record_dt,sum,details
0,1,2024-01-01,-4.25,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312290754, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1318| BCC: 1; IPS: 1.1142"
1,2,2024-01-01,0.09,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 4.25 EUR"
2,3,2024-01-01,0.11,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 5.59 EUR"
3,4,2024-01-01,-5.59,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312291925, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1306| BCC: 1; IPS: 1.1142"
4,5,2024-01-01,0.13,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 6.59 EUR"
...,...,...,...,...
4052,4053,2024-12-30,-7.00,"Retail. Номер устройства в ПЦ, 27.12.2024 00:00:00, PRT, LISBOA, FABRICA COFFE, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0586| BCC: 1; IPS: 1.0434"
4053,4054,2024-12-30,-7.60,"Retail. Номер устройства в ПЦ, 27.12.2024 00:00:00, PRT, LISBOA, FABRICA COFFE, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0592| BCC: 1; IPS: 1.0434"
4054,4055,2024-12-30,1098.48,"Покупка иностранной валюты клиентом розницы, Коренблюм Герман Владимирович, БИН 920404050799. Заявка №11013165686. Дата вал. 30.12.2024. Покупка EUR, продажа KZT. Курс сделки 546.21. Учетный курс 543.67. Конвертация со счета KZ348562204134322019 на счет KZ938562204234322234 через систему BCC.KZ"
4055,4056,2024-12-31,-4.80,"Retail. Номер устройства в ПЦ, 28.12.2024 00:00:00, PRT, LISBOA, REST LULLY 1661, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0604| BCC: 1; IPS: 1.0445"


In [72]:
#parsing details
first_word = r'(^\w+)'
forex = r'^Покупка иностранной валюты'
atm = r'^Снятие наличных АТМ'
transfer = r'^Перевод \(списание\)'
returns = r'^Прочие зачисления на карту \(credit\)'
retail_regex = r'^(?:[^,]*,){4}([^,]*)'
retail_loc_regex = r'^[^,]*,[^,]*,\s*([^,]*,[^,]*)'
retail_datetime_regex = r'^[^,]*,\s*(\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2})'

def check_retail(row):
    if re.match(first_word, row['details']).group(0) == 'Retail':
        row['pos_loc'] = re.search(retail_loc_regex, row['details']).group(1)
        row['pos'] = re.search(retail_regex, row['details']).group(1)
        row['transaction_dt'] = re.search(retail_datetime_regex, row['details']).group(1)
        row['category_1'] = 'retail'
    elif re.match(forex, row['details']):
        row['category_1'] = 'forex'
    elif re.match(atm, row['details']):
        row['category_1'] = 'atm'
    elif re.match(transfer, row['details']):
        row['category_1'] = 'transfer'
    elif re.match(returns, row['details']):
        row['category_1'] = 'returns'
    else:
        row['category_1'] = 'unallocated'
    return row

df_euro = df_euro.apply(check_retail, axis=1)

,category_1,details,id,pos,pos_loc,record_dt,sum,transaction_dt
0,retail,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312290754, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1318| BCC: 1; IPS: 1.1142",1,BOLT.EU/O/2312290754,"EST, Tallinn",2024-01-01,-4.25,29.12.2023 00:00:00
1,unallocated,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 4.25 EUR",2,NaN,NaN,2024-01-01,0.09,NaN
2,unallocated,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 5.59 EUR",3,NaN,NaN,2024-01-01,0.11,NaN
3,retail,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312291925, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1306| BCC: 1; IPS: 1.1142",4,BOLT.EU/O/2312291925,"EST, Tallinn",2024-01-01,-5.59,29.12.2023 00:00:00
4,unallocated,"Учет вознаграждений по CashBack. Дата 29.12.2023 00:00:00, сумма 6.59 EUR",5,NaN,NaN,2024-01-01,0.13,NaN
...,...,...,...,...,...,...,...,...
4052,retail,"Retail. Номер устройства в ПЦ, 27.12.2024 00:00:00, PRT, LISBOA, FABRICA COFFE, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0586| BCC: 1; IPS: 1.0434",4053,FABRICA COFFE,"PRT, LISBOA",2024-12-30,-7.00,27.12.2024 00:00:00
4053,retail,"Retail. Номер устройства в ПЦ, 27.12.2024 00:00:00, PRT, LISBOA, FABRICA COFFE, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0592| BCC: 1; IPS: 1.0434",4054,FABRICA COFFE,"PRT, LISBOA",2024-12-30,-7.60,27.12.2024 00:00:00
4054,forex,"Покупка иностранной валюты клиентом розницы, Коренблюм Герман Владимирович, БИН 920404050799. Заявка №11013165686. Дата вал. 30.12.2024. Покупка EUR, продажа KZT. Курс сделки 546.21. Учетный курс 543.67. Конвертация со счета KZ348562204134322019 на счет KZ938562204234322234 через систему BCC.KZ",4055,NaN,NaN,2024-12-30,1098.48,NaN
4055,retail,"Retail. Номер устройства в ПЦ, 28.12.2024 00:00:00, PRT, LISBOA, REST LULLY 1661, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0604| BCC: 1; IPS: 1.0445",4056,REST LULLY 1661,"PRT, LISBOA",2024-12-31,-4.80,28.12.2024 00:00:00


In [73]:
# Regular expressions to extract date and retail sum directly in the loop
cashback_date_regex = r'Дата (\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2})'
retail_sum_regex = r'сумма ([\d.]+)'

# Step 2: Track indices of cashback rows that have matching retail transactions
matched_cashback_indices = []

# Step 3: Process cashback rows, find matching Retail rows, and add cashback sum
for index, row in df_euro.iterrows():
    if "Учет вознаграждений по CashBack" in row['details']:
        # Extract the cashback date and retail transaction sum directly
        cashback_date_match = re.search(cashback_date_regex, row['details'])
        retail_sum_match = re.search(retail_sum_regex, row['details'])
        
        if cashback_date_match and retail_sum_match:
            cashback_date = cashback_date_match.group(1)
            retail_amount = float(retail_sum_match.group(1))
            cashback_amount = row['sum']
            
            # Find matching Retail row by 'transaction_dt' and 'sum'
            matching_retail_index = df_euro[(df_euro['transaction_dt'] == cashback_date) & 
                                            (df_euro['sum'] == -retail_amount)].index
            
            # If match found, update cashback_sum in the Retail row and mark cashback row for deletion
            if not matching_retail_index.empty:
                df_euro.loc[matching_retail_index, 'cashback_sum'] = cashback_amount
                matched_cashback_indices.append(index)

# Step 4: Remove matched cashback rows
df_euro.drop(matched_cashback_indices, inplace=True)

# Reset index after dropping rows (optional)
df_euro.reset_index(drop=True, inplace=True)

,category_1,details,id,pos,pos_loc,record_dt,sum,transaction_dt,cashback_sum
0,retail,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312290754, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1318| BCC: 1; IPS: 1.1142",1,BOLT.EU/O/2312290754,"EST, Tallinn",2024-01-01,-4.25,29.12.2023 00:00:00,0.09
1,retail,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312291925, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1306| BCC: 1; IPS: 1.1142",4,BOLT.EU/O/2312291925,"EST, Tallinn",2024-01-01,-5.59,29.12.2023 00:00:00,0.11
2,retail,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312292104, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1308| BCC: 1; IPS: 1.1087",6,BOLT.EU/O/2312292104,"EST, Tallinn",2024-01-01,-6.59,29.12.2023 00:00:00,0.13
3,retail,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312291705, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1322| BCC: 1; IPS: 1.1142",7,BOLT.EU/O/2312291705,"EST, Tallinn",2024-01-01,-4.54,29.12.2023 00:00:00,0.09
4,retail,"Retail. Номер устройства в ПЦ, 29.12.2023 00:00:00, EST, Tallinn, BOLT.EU/O/2312291958, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.1318| BCC: 1; IPS: 1.1142",10,BOLT.EU/O/2312291958,"EST, Tallinn",2024-01-01,-4.25,29.12.2023 00:00:00,0.09
...,...,...,...,...,...,...,...,...,...
2263,retail,"Retail. Номер устройства в ПЦ, 27.12.2024 00:00:00, PRT, LISBOA, FABRICA COFFE, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0586| BCC: 1; IPS: 1.0434",4053,FABRICA COFFE,"PRT, LISBOA",2024-12-30,-7.00,27.12.2024 00:00:00,NaN
2264,retail,"Retail. Номер устройства в ПЦ, 27.12.2024 00:00:00, PRT, LISBOA, FABRICA COFFE, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0592| BCC: 1; IPS: 1.0434",4054,FABRICA COFFE,"PRT, LISBOA",2024-12-30,-7.60,27.12.2024 00:00:00,NaN
2265,forex,"Покупка иностранной валюты клиентом розницы, Коренблюм Герман Владимирович, БИН 920404050799. Заявка №11013165686. Дата вал. 30.12.2024. Покупка EUR, продажа KZT. Курс сделки 546.21. Учетный курс 543.67. Конвертация со счета KZ348562204134322019 на счет KZ938562204234322234 через систему BCC.KZ",4055,NaN,NaN,2024-12-30,1098.48,NaN,NaN
2266,retail,"Retail. Номер устройства в ПЦ, 28.12.2024 00:00:00, PRT, LISBOA, REST LULLY 1661, Карта: 462818******4620 Валюты:EUR-USD| IPS: 1.0604| BCC: 1; IPS: 1.0445",4056,REST LULLY 1661,"PRT, LISBOA",2024-12-31,-4.80,28.12.2024 00:00:00,NaN


### Preparing BCC Tenge DF

In [74]:
df_tenge

Транзакция жасалған карта/Transaction for Card/Транзакции по карте №462818******4620  \
                                          Есепшот бойынша күн Posting Date Дата по счёту   
                                          Есепшот бойынша күн Posting Date Дата по счёту   
0                                                                             01.01.2024   
1                                                                             01.01.2024   
2                                                                             02.01.2024   
3                                                                             03.01.2024   
4                                                                             04.01.2024   
..                                                                                   ...   
236                                                                           24.12.2024   
237                                                                           23.12.2024   
238                                                                           28.12.2024   
239                                                                           28.12.2024   
240                                                                           30.12.2024   

                                                \
    Операция күні Operation Date Дата операции   
    Операция күні Operation Date Дата операции   
0                                   01.01.2024   
1                                   01.01.2024   
2                                   02.01.2024   
3                                   03.01.2024   
4                                   04.01.2024   
..                                         ...   
236                                 21.12.2024   
237                                 23.12.2024   
238                                 28.12.2024   
239                                 28.12.2024   
240                                 30.12.2024   

                                                                                                                                                                                                                                                                                                                                                        \
                                                                                                                                                                                                                                                                                              Операция сипаттамасы Operation Details Описание операции   
                                                                                                                                                                                                                                                                                              Операция сипаттамасы Operation Details Описание операции   
0                                                                                 Перевод с карты 446375******1579 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Самойленко Ксения Владимировна   
1                                                                                    Перевод с карты 446375******8122 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Соколов Александр Сергеевич   
2                                                                                Перевод с карты 446375******2183 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безнал

In [75]:
#renaming headers
df_tenge.columns = [' '.join(col).strip() for col in df_tenge.columns.values]

df_tenge = df_tenge.rename(
    columns={
        df_tenge.columns[0]: 'record_dt',
        df_tenge.columns[1]: 'transaction_dt',
        df_tenge.columns[2]: 'details',
        df_tenge.columns[3]: 'sum_in_currency',
        df_tenge.columns[4]: 'currency',
        df_tenge.columns[5]: 'fee',
        df_tenge.columns[6]: 'total_sum',
        df_tenge.columns[7]: 'cashback',
    }
    
)
df_tenge.head(10)

df_tenge['sum_in_currency'] = df_tenge['sum_in_currency'].str.replace(r'\s+', '', regex=True)
df_tenge['sum_in_currency'] = pd.to_numeric(df_tenge['sum_in_currency'])

df_tenge['total_sum'] = df_tenge['total_sum'].str.replace(r'\s+', '', regex=True)
df_tenge['total_sum'] = pd.to_numeric(df_tenge['total_sum'])

df_tenge['fee'] = df_tenge['fee'].str.replace(r'\s+', '', regex=True)
df_tenge['fee'] = pd.to_numeric(df_tenge['fee'])

df_tenge['cashback'] = df_tenge['cashback'].str.replace(r'\s+', '', regex=True)
df_tenge['cashback'] = pd.to_numeric(df_tenge['cashback'])

df_tenge['record_dt'] = pd.to_datetime(df_tenge['record_dt'], format='%d.%m.%Y')
df_tenge['transaction_dt'] = pd.to_datetime(df_tenge['transaction_dt'], format='%d.%m.%Y')
df_tenge

,record_dt,transaction_dt,details,sum_in_currency,currency,fee,total_sum,cashback
0,2024-01-01,2024-01-01,Перевод с карты 446375******1579 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Самойленко Ксения Владимировна,513400.00,KZT,0.0,513400.00,NaN
1,2024-01-01,2024-01-01,Перевод с карты 446375******8122 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Соколов Александр Сергеевич,308000.00,KZT,0.0,308000.00,NaN
2,2024-01-02,2024-01-02,Перевод с карты 446375******2183 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Кудреватых Александр Валерьевич,225682.00,KZT,0.0,225682.00,NaN
3,2024-01-03,2024-01-03,"Покупка иностранной валюты клиентом розницы, Коренблюм Герман Владимирович, БИН 920404050799. Зявка №161092700. Дата вал. 03.01.2024. Покупка EUR, продажа KZT. Курс сделки 501.01. Учетный курс 502.24. Конвертация со счета KZ348562204134322019 на счет KZ938562204234322234 через систему BCC.KZ Плательщик: Коренблюм Герман Владимирович",1062141.20,KZT,0.0,-1062141.20,NaN
4,2024-01-04,2024-01-04,Перевод на счет KZ348562204134322019,67940.00,KZT,0.0,67940.00,NaN
...,...,...,...,...,...,...,...,...
236,2024-12-24,2024-12-21,"Retail. Номер устройства в ПЦ, 21.12.2024 00:00:00, SRB, BEOGRAD, BISTRO, Карта: 462818******4620 Валюты:RSD-USD| IPS: 0.0091| BCC: 1; IPS: 0.0089 Плательщик: Коренблюм Герман Владимирович (RRN:435612004472, Auth code:534103)",1880.00,RSD,0.0,-8980.08,NaN
237,2024-12-23,2024-12-23,"Покупка иностранной валюты клиентом розницы, Коренблюм Герман Владимирович, БИН 920404050799. Заявка №10929801805. Дата вал. 23.12.2024. Покупка EUR, продажа KZT. Курс сделки 545.05. Учетный курс 544.34. Конвертация со счета KZ348562204134322019 на счет KZ938562204234322234 через систему BCC.KZ Плательщик: Коренблюм Герман Владимирович",400000.00,KZT,0.0,-400000.00,NaN
238,2024-12-28,2024-12-28,Перевод на счет KZ348562204134322019,241245.00,KZT,0.0,241245.00,NaN
239,2024-12-28,2024-12-28,Перевод кешбэка на карту Плательщик: Центральное отделение №0102,135087.51,KZT,0.0,135087.51,NaN


In [12]:
df_tenge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   record_dt        241 non-null    datetime64[ns]
 1   transaction_dt   241 non-null    datetime64[ns]
 2   details          241 non-null    object        
 3   sum_in_currency  241 non-null    float64       
 4   currency         241 non-null    object        
 5   fee              241 non-null    float64       
 6   total_sum        241 non-null    float64       
 7   cashback         117 non-null    float64       
dtypes: datetime64[ns](2), float64(4), object(2)
memory usage: 15.2+ KB


In [13]:
df_tenge.head(3)

,record_dt,transaction_dt,details,sum_in_currency,currency,fee,total_sum,cashback
0,2024-01-01,2024-01-01,Перевод с карты 446375******1579 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Самойленко Ксения Владимировна,513400.0,KZT,0.0,513400.0,NaN
1,2024-01-01,2024-01-01,Перевод с карты 446375******8122 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Соколов Александр Сергеевич,308000.0,KZT,0.0,308000.0,NaN
2,2024-01-02,2024-01-02,Перевод с карты 446375******2183 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Кудреватых Александр Валерьевич,225682.0,KZT,0.0,225682.0,NaN


### Prepare Caixa Euro DF

### Merging and Mapping

In [65]:
# Load excel file with category mapping
df_cat_mapping = pd.read_excel("category_mapping.xlsx", sheet_name="Sheet1")

cat_mapping = {}
for _, row in df_cat_mapping.iterrows():
    cat = row['category']
    pos = row['pos']
    if cat not in cat_mapping:
        cat_mapping[cat] = []
    cat_mapping[cat].append(pos)

def assign_category(row):
    for category, keywords in cat_mapping.items():
        if any(keyword in row['details'] for keyword in keywords):
            return category
    return 'unallocated'

# Apply the function to create the 'category' column
df_euro.loc[df_euro['category_1'] == 'retail', 'category_2'] = df_euro[df_euro['category_1'] == 'retail'].apply(assign_category, axis=1)

unallocated_count = df_euro[df_euro['category_2'] == 'unallocated'].shape[0]
print("Number of unallocated rows:", unallocated_count)


/Users/korenblyum/Dev/budget/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Number of unallocated rows: 133


### Reports

In [66]:
#df_euro[df_euro['category_2'] == 'clothes'].sort_values(by='sum').head(10)

monthly_sum = df_euro.groupby([df_euro['record_dt'].dt.to_period('M'), 'category_2'])['sum'].sum().reset_index()

# Pivot the DataFrame to have months as columns
pivot_table = monthly_sum.pivot(index='category_2', columns='record_dt', values='sum').fillna(0)

# Convert the PeriodIndex to a string for clarity
pivot_table.columns = pivot_table.columns.astype(str)

# Display the pivot table
pivot_table

record_dt,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
category_2,,,,,,,,,,,,
amazon,-1316.18,-590.74,-196.99,-581.50,-4.99,-139.41,-620.91,-738.91,-4.99,-157.82,-148.04,-407.48
clothes,-167.90,-783.99,-983.64,-1261.63,-588.35,-634.08,-566.06,-200.59,-669.03,-232.80,-590.29,-984.12
eat_out,-478.50,-521.72,-729.09,-696.12,-678.44,-362.30,-367.20,-489.75,-286.80,-224.80,-562.02,-182.80
food_order,-155.58,-86.44,-161.47,-183.41,-141.90,-8.50,-27.85,-45.26,-139.70,-111.02,-163.91,-114.42
grocery,-506.22,-798.68,-741.63,-868.12,-662.45,-764.29,-913.45,-665.66,-538.47,-859.25,-612.42,-747.32
home,-2859.34,-805.50,-90.23,-500.60,-150.69,-42.10,-86.08,-501.21,-489.84,-599.32,-283.61,-231.77
misc,-790.23,-1059.53,-494.44,-453.88,-1881.95,-548.31,-269.09,-427.46,-562.27,-103.52,-38.00,-747.43
pet,-440.66,0.00,0.00,-185.32,0.00,0.00,0.00,-113.77,-240.70,-110.60,0.00,0.00
subscriptions,-35.00,-50.00,-72.13,-72.13,-83.63,-72.13,-57.13,-56.14,-56.14,-145.50,-72.04,-131.66
